All the credits goes to Sebastian Lague.
[His video](https://www.youtube.com/watch?v=hfMk-kjRv4c&t=189s&ab_channel=SebastianLague)

# General Idea

### We are trying to understand if an imaginary fruit is poisonous or not by their spike and spot sizes.

### There will be 2 inputs for spike and spot and 2 outputs poisonous or non-poisonous

### If output1 > output2 then its non-poisonous otherwise its poisonous

<!-- ![](../assets/ss-1.png) -->

## First we are trying to create a simple network as shown below

<img src="../assets/ss-1.png" height="300">

! Model below does not scale.

In [95]:
# Weight values connect each input to each output.
weight_1_1: float
weight_2_1: float

weight_1_2: float
weight_2_2: float

# Bias values
bias_1: float
bias_2: float


def Classify(input_1: float, input_2: float) -> int:
    global weight_1_1
    global weight_2_1
    global weight_1_2
    global weight_2_2
    global bias_1
    global bias_2

    output_1: float = (input_1 * weight_1_1) + (input_2 * weight_2_1) + bias_1
    output_2: float = (input_1 * weight_1_2) + (input_2 * weight_2_2) + bias_2

    return 0 if output_1 > output_2 else 1


## Structure of Hidden Layers

<img src="../assets/ss-2.png" height="300">

### We are just doing Linear Regression. We need an Activation Function in order to get a weighted input between 0 - 1

#### Here is a simple Activation Function.
<img src="../assets/ss-3.png" height="300">

#### Here is a more complex one. (Sigmoid Function)
<img src="../assets/ss-4.png" height="300">

In [96]:
# from dataclasses import dataclass
from typing import List
import math
import random

[Sigmoid Function](https://en.wikipedia.org/wiki/Sigmoid_function)

In [97]:
# Sigmoid function
def ActivationFunction(weightedInput: float) -> float:
    return 1 / (1 + math.exp(-weightedInput))


## We will implement a Gradiant Cost method.
-- Add Further info here!

In [98]:
class Layer:
    numNodesIn: int
    numNodesOut: int
    costGradientW: List[List[float]]
    costGradientB: List[float]
    weights: List[List[float]]
    biases: List[float]

    def __init__(self, numNodesIn: int, numNodesOut: int):
        self.numNodesIn = numNodesIn
        self.numNodesOut = numNodesOut

        self.weights = [[0.0 for i in range(numNodesOut)] for j in range(numNodesIn)]
        self.biases = [0.0 for i in range(numNodesOut)]

        self.InitializeRandomWeights()

    def InitializeRandomWeights(self):
        for nodeIn in range(self.numNodesIn):
            for nodeOut in range(self.numNodesOut):
                randomValue = random.random() * -1
                self.weights[nodeIn][nodeOut] = randomValue / math.sqrt(self.numNodesIn)

    def CalculateOutputs(self, inputs: List[float]) -> List[float]:
        activations: List[float]

        for nodeOut in range(self.numNodesOut):
            weightedInput = self.biases[nodeOut]

            for nodeIn in range(self.numNodesIn):
                weightedInput += inputs[nodeIn] * self.weights[nodeIn][nodeOut]

            activations[nodeOut] = ActivationFunction(weightedInput)

        return activations

    def NodeCost(outputActivation: float, expectedOutput: float) -> float:
        error = outputActivation - expectedOutput
        return error * error

    def ApplyGradients(self, learningRate: float):
        for nodeOut in range(self.numNodesOut):
            self.biases[nodeOut] -= self.costGradientB[nodeOut] * learningRate

            for nodeIn in range(self.numNodesIn):
                self.weights[nodeIn][nodeOut] -= self.costGradientW[nodeIn][nodeOut] * learningRate

    def NodeCostDerivative(outputActivation: float, expectedOutput: float) -> float:
        return 2 * (outputActivation - expectedOutput)

    def ActivationDerivative(weightedInput: float) -> float:
        activation = ActivationFunction(weightedInput)
        return activation * (1 - activation)

    def CalculateOutputLayerNodeValues(self, expectedOutputs: List[float]) -> float:
        nodeValues: List[float]

        for i in range(len(expectedOutputs)):
            costDerivative = self.NodeCostDerivative(self.activations[i], expectedOutputs[i])
            activationDerivative = self.ActivationDerivative(self.weightedInputs[i])
            nodeValues[i] = costDerivative * activationDerivative

        return nodeValues

    def UpdateGradients(self, nodeValues: List[float]):
        for nodeOut in range(self.numNodesOut):
            for nodeIn in range(self.numNodesIn):
                self.costGradientW[nodeIn][nodeOut] += self.inputs[nodeIn] * nodeValues[nodeOut]

In [99]:
class NeuralNetwork:
    layers: List[Layer]

    def __init__(self, layerSizes: List[int]):
        self.layers = []

        for i in range(len(layerSizes) - 1):
            self.layers.append(Layer(layerSizes[i], layerSizes[i + 1]))

    def CalculateOutputs(self, inputs: List[float]) -> List[float]:
        for layer in self.layers:
            inputs = layer.CalculateOutputs(inputs)

        return inputs

    def Classify(self, inputs: List[float]) -> int:
        outputs = self.CalculateOutputs(inputs)
        return 0 if outputs[0] > outputs[1] else 1

    def CalculateCost(self, inputs: List[float], expectedOutputs: List[float]) -> float:
        outputs = self.CalculateOutputs(inputs)
        outputLayer: Layer = self.layers[-1]
        cost = 0.0

        for nodeOut in range(len(outputs)):
            cost += outputLayer.NodeCost(outputs[nodeOut], expectedOutputs[nodeOut])

        return cost

    def Learn(self, inputs: List[float], expectedOutputs: List[float], learningRate: float):
        h: float = 0.0001
        originalCost: float = self.CalculateCost(inputs, expectedOutputs)

        for layer in self.layers:
            for nodeIn in range(layer.numNodesIn):
                for nodeOut in range(layer.numNodesOut):
                    layer.weights[nodeIn][nodeOut] += h
                    deltaCost = self.CalculateCost(inputs, expectedOutputs) - originalCost
                    layer.weights[nodeIn][nodeOut] -= h
                    layer.costGradientW[nodeIn][nodeOut] = deltaCost / h

            for biasIndex in range(len(layer.biases)):
                layer.biases[biasIndex] += h
                deltaCost = self.CalculateCost(inputs, expectedOutputs) - originalCost
                layer.biases[biasIndex] -= h
                layer.costGradientB[biasIndex] = deltaCost / h

        for layer in self.layers:
            layer.ApplyGradients(learningRate)

    def UpdateAllGradients(self, inputs: List[float], expectedOutputs: List[float]):
        self.CalculateOutputs(inputs)

        outputLayer: Layer = self.layers[-1]
        nodeValues = outputLayer.Cal

In [100]:
network = NeuralNetwork([2, 3, 2])